# 7장 데이터 합치기/변형

In [3]:
from pandas import Series, DataFrame
import pandas as pd

In [4]:
df1 = DataFrame({
        'key': list('bbacaab'),
        'data1': range(7)
    })

In [10]:
df1 = df1[['key', 'data1']]
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [6]:
df2 = DataFrame({
        'key': list('abd'),
        'data2': range(3)
    })

In [12]:
df2 = df2[['key', 'data2']]
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [13]:
pd.merge(df1, df2)[['key', 'data1', 'data2']]

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [14]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [16]:
df1.columns = ['key1', 'data1']

In [18]:
df2.columns = ['key2', 'data2']

In [21]:
df = pd.merge(df1, df2, left_on='key1', right_on='key2')
df.drop('key2', axis=1)

,key1,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


#### 연습

(데자뷰?!) data/pydata-book/movielens 폴더에는 영화 평점과 관련된 users.dat, ratings.dat, movies.dat 세 개의 파일이 있다.

이 파일을 데이터프레임으로 읽어들여 다음을 수행

1. ratings와 users 합치기
1. movies, ratings 합치기
1. ratings, movies, users 합치기

In [24]:
users = pd.read_csv(
    'data/pydata-book/movielens/users.dat',
    sep='::', engine='python', 
    names=['user_id', '성별', '나이', '직업', '지역']
)

In [25]:
users[:3]

,user_id,성별,나이,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [30]:
ratings = pd.read_csv(
    'data/pydata-book/movielens/ratings.dat',
    sep='::', engine='python',
    names=['user', 'movie', '평점', 'timestamp']
)

In [31]:
ratings[:3]

,user,movie,평점,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [ ]:
rating_user = pd.merge(
    ratings, users, left_on='user', right_on='user_id')

In [ ]:
rating_user = rating_user.drop('user_id', axis=1)

In [36]:
rating_user[:3]

,user,movie,평점,timestamp,성별,나이,직업,지역
0,1,1193,5,978300760,F,1,10,48067
1,1,661,3,978302109,F,1,10,48067
2,1,914,3,978301968,F,1,10,48067


In [40]:
movies = pd.read_csv(
    'data/pydata-book/movielens/movies.dat',
    sep='::', engine='python',
    names=['mid', '제목', '장르']
)

In [41]:
movies[:3]

,mid,제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [44]:
movies_ratings = pd.merge(
    movies, ratings, left_on='mid', right_on='movie')

In [47]:
movies_ratings = movies_ratings.drop('movie', axis=1)
movies_ratings[:3]

,mid,제목,장르,user,평점,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496


In [49]:
df = pd.merge(
    ratings, movies, left_on='movie', right_on='mid')
df = pd.merge(df, users, left_on='user', right_on='user_id')

In [50]:
df[:3]

,user,movie,평점,timestamp,mid,제목,장르,user_id,성별,나이,직업,지역
0,1,1193,5,978300760,1193,One Flew Over the Cuckoo's Nest (1975),Drama,1,F,1,10,48067
1,1,661,3,978302109,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,F,1,10,48067
2,1,914,3,978301968,914,My Fair Lady (1964),Musical|Romance,1,F,1,10,48067


### 색인 합치기

In [52]:
df1 = df1.set_index('key1')

In [54]:
df1

,data1
key1,
b,0
b,1
a,2
c,3
a,4
a,5
b,6


In [53]:
df2 = df2.set_index('key2')

In [59]:
pd.merge(df1, df2, left_index=True, right_index=True)

,data1,data2
a,2,0
a,4,0
a,5,0
b,0,1
b,1,1
b,6,1


색인은 더 이상 열이 아닙니다.

In [58]:
'key1' in df1

False

In [61]:
df2 = df2.reset_index()

In [62]:
pd.merge(df1, df2, left_index=True, right_on='key2')

,data1,key2,data2
1,0,b,1
1,1,b,1
1,6,b,1
0,2,a,0
0,4,a,0
0,5,a,0


In [63]:
df2 = df2.set_index('key2')

In [64]:
df1.join(df2)

,data1,data2
a,2,0.0
a,4,0.0
a,5,0.0
b,0,1.0
b,1,1.0
b,6,1.0
c,3,NaN


#### 연습

data/food_order.xlsx 와 data/weather.xlsx 를 각각 데이터프레임으로 구성하고, 두 개의 데이터프레임을 하나로 합쳐 변수 '식당날씨'에 저장한다.

1. date 열을 기준으로 합친다.
1. 식당과 날씨 자료의 date 열을 각각 색인으로 설정하고, 색인을 기준으로 합친다.
1. 식당을 서울날씨와 합친다.

In [65]:
식당 = pd.read_excel('data/food_order.xlsx')

In [66]:
날씨 = pd.read_excel('data/weather.xlsx')

In [67]:
서울날씨 = 날씨[날씨.location == 'seoul']

In [68]:
식당 = 식당.set_index('date')

In [69]:
서울날씨 = 서울날씨.set_index('date')

In [70]:
식당날씨 = 식당.join(서울날씨)

In [72]:
식당날씨[['menu', 'weather']][:3]

,menu,weather
date,,
2013-08-06,훈제연어벤또(200개),구름많음/안개/천둥번개/비
2013-08-06,뚝배기 우거지 갈비탕,구름많음/안개/천둥번개/비
2013-08-06,판모밀정식,구름많음/안개/천둥번개/비


### 축 따라 이어붙이기

pandas.concat

In [73]:
import baseball

In [74]:
파일목록 = []
for 연도 in range(2013, 2016):
    경로 = 'data/NC Dinos {}.xlsx'.format(연도)
    파일목록.append(경로)

In [75]:
NC시즌 = baseball.from_files(파일목록)

색인방향(axis=0) ~ "세로"로 이어붙이기

In [79]:
pd.concat(NC시즌)

,타석,안타,홈런
선수명,,,
모창민,436,109,12
이호준,508,123,20
김종호,546,129,0
나성범,458,98,14
조영훈,426,107,6
이현곤,161,38,0
이상호,138,31,0
강진성,3,1,0
조평호,86,21,2


열 방향(axis=1, "가로")로 이어붙이기

In [80]:
홈런 = [NC시즌[0].홈런, NC시즌[1].홈런, NC시즌[2].홈런]

In [81]:
pd.concat(홈런, axis=1)

,홈런,홈런,홈런
강구성,0.0,NaN,0.0
강민국,NaN,0.0,0.0
강진성,0.0,NaN,NaN
권희동,NaN,7.0,NaN
김동건,1.0,NaN,NaN
김성욱,0.0,1.0,NaN
김종찬,0.0,NaN,NaN
김종호,0.0,NaN,4.0
김준완,NaN,0.0,0.0
김태군,NaN,NaN,6.0


#### 연습

NC 타자들의 지난 3년간 누적 타율 계산

In [ ]:
홈런열 = [시즌.홈런 for 시즌 in NC시즌]
시즌홈런 = pd.concat(홈런열, axis=1)
홈런합계 = 시즌홈런.sum(1)

In [86]:
시즌안타 = pd.concat([시즌.안타 for 시즌 in NC시즌], axis=1)
안타합계 = 시즌안타.sum(1)

In [87]:
시즌타석 = pd.concat([시즌.타석 for 시즌 in NC시즌], axis=1)
타석합계 = 시즌타석.sum(1)

In [88]:
누적타율 = (안타합계 + 홈런합계) / 타석합계

In [89]:
누적타율.sort_values(ascending=False)[:3]

테임즈    0.375113
강진성    0.333333
나성범    0.316213
dtype: float64

## 계층색인

In [94]:
for 연도, 프레임 in zip(range(2013, 2016), NC시즌):
    프레임['시즌'] = 연도

In [96]:
NC13 = NC시즌[0].reset_index()

In [ ]:
NC시즌 = pd.concat(NC시즌)

In [106]:
NC시즌 = NC시즌.set_index('시즌', append=True)

계층 색인의 위치 설정

In [110]:
NC시즌 = NC시즌.swaplevel('선수명', '시즌')

In [111]:
NC시즌.ix[2013]

,타석,안타,홈런
선수명,,,
모창민,436,109,12
이호준,508,123,20
김종호,546,129,0
나성범,458,98,14
조영훈,426,107,6
이현곤,161,38,0
이상호,138,31,0
강진성,3,1,0
조평호,86,21,2


"나성범 선수의 전체 시즌 기록 선택"

In [115]:
나테 = NC시즌.swaplevel(0,1).ix[['나성범', '테임즈']]

## 재형성과 피벗

In [121]:
나테.unstack()

타석                  안타                  홈런            
시즌    2013   2014   2015  2013   2014   2015  2013  2014  2015
선수명                                                           
나성범  458.0  536.0  622.0  98.0  157.0  184.0  14.0  30.0  28.0
테임즈    NaN  514.0  595.0   NaN  152.0  180.0   NaN  37.0  47.0

In [123]:
나테.stack()

선수명  시즌      
나성범  2013  타석    458
           안타     98
           홈런     14
     2014  타석    536
           안타    157
           홈런     30
     2015  타석    622
           안타    184
           홈런     28
테임즈  2014  타석    514
           안타    152
           홈런     37
     2015  타석    595
           안타    180
           홈런     47
dtype: int64

대상 색인 단계 선택

In [127]:
나테.unstack('선수명')

타석            안타           홈런      
선수명     나성범    테임즈    나성범    테임즈   나성범   테임즈
시즌                                          
2013  458.0    NaN   98.0    NaN  14.0   NaN
2014  536.0  514.0  157.0  152.0  30.0  37.0
2015  622.0  595.0  184.0  180.0  28.0  47.0

#### 연습

계층색인과 stack/unstack을 활용해 3년간의 누적 타율 계산

In [134]:
타자별 = NC시즌.unstack(0)

In [135]:
타석합계 = 타자별['타석'].sum(1)
안타합계 = 타자별['안타'].sum(1)
홈런합계 = 타자별['홈런'].sum(1)

In [136]:
타율 = (안타합계 + 홈런합계) / 타석합계

In [137]:
타율.sort_values(ascending=False)[:3]

선수명
테임즈    0.375113
강진성    0.333333
나성범    0.316213
dtype: float64

In [140]:
NC시즌.swaplevel(0,1).sort_index()

타석   안타  홈런
선수명  시즌                
강구성  2013    2    0   0
     2015    5    1   0
강민국  2014    3    0   0
     2015    2    0   0
강진성  2013    3    1   0
권희동  2014  252   63   7
김동건  2013   17    2   1
김성욱  2013    4    1   0
     2014   26    4   1
김종찬  2013    7    1   0
김종호  2013  546  129   0
     2015  486  125   4
김준완  2014    5    2   0
     2015   54   10   0
김태군  2015  474  107   6
김태우  2014    5    1   0
김태진  2015    3    0   0
나성범  2013  458   98  14
     2014  536  157  30
     2015  622  184  28
노진혁  2014   16    3   1
마낙길  2013   31    9   0
     2014    2    0   0
모창민  2013  436  109  12
     2014  468  110  16
     2015  239   62   6
박명환* 2014    1    0   0
박민우  2013   48   11   0
     2014  491  124   1
     2015  617  158   3
박정준  2013  195   41   4
     2015   24    8   0
손시헌  2014  361   90   5
     2015  508  108  13
용덕한  2015   49   12   0
이상호  2013  138   31   0
이승호  2013    1    0   0
이종욱  2014  495  126   6
     2015  504  118   5
이창섭  2013    5    0   0
     2015    8    0   0
이현곤  2013  161   38   0
이호준  2013  508  123  20
     2014  500  115  23
     2015  518  132  24
조영훈  2013  426  107   6
     2014  124   29   6
     2015  146   35   8
조평호  2013   86   21   2
     2014    2    0   0
     2015   22    8   1
지석훈  2013  356   68   3
     2014  238   58   6
     2015  466  111  11
차화준  2013  121   24   1
최재원  2014    2    0   0
     2015  102   21   2
테임즈  2014  514  152  37
     2015  595  180  47
허준   2014   36    5   0

In [142]:
NC시즌.to_excel('NC시즌.xlsx')

In [143]:
NC시즌 = pd.read_excel('NC시즌.xlsx')

In [147]:
NC시즌.fillna(method='ffill').set_index(['시즌', '선수명'])

타석   안타  홈런
시즌     선수명               
2013.0 모창민   436  109  12
       이호준   508  123  20
       김종호   546  129   0
       나성범   458   98  14
       조영훈   426  107   6
       이현곤   161   38   0
       이상호   138   31   0
       강진성     3    1   0
       조평호    86   21   2
       박민우    48   11   0
       박정준   195   41   4
       이승호     1    0   0
       마낙길    31    9   0
       김성욱     4    1   0
       강구성     2    0   0
       지석훈   356   68   3
       김종찬     7    1   0
       차화준   121   24   1
       김동건    17    2   1
       이창섭     5    0   0
2014.0 테임즈   514  152  37
       나성범   536  157  30
       박민우   491  124   1
       손시헌   361   90   5
       지석훈   238   58   6
       이호준   500  115  23
       권희동   252   63   7
       모창민   468  110  16
       이종욱   495  126   6
       김준완     5    2   0
       최재원     2    0   0
       마낙길     2    0   0
       조영훈   124   29   6
       김태우     5    1   0
       박명환*    1    0   0
       조평호     2    0   0
       강민국     3    0   0
       허준     36    5   0
       김성욱    26    4   1
       노진혁    16    3   1
2015.0 테임즈   595  180  47
       나성범   622  184  28
       박민우   617  158   3
       이호준   518  132  24
       김종호   486  125   4
       조영훈   146   35   8
       손시헌   508  108  13
       이종욱   504  118   5
       모창민   239   62   6
       지석훈   466  111  11
       용덕한    49   12   0
       최재원   102   21   2
       박정준    24    8   0
       김태군   474  107   6
       조평호    22    8   1
       이창섭     8    0   0
       김준완    54   10   0
       강구성     5    1   0
       강민국     2    0   0
       김태진     3    0   0

#### 연습

식당 데이터프레임에 대해 다음을 수행

1. corner, dine_type을 색인으로 설정
1. 아침 시간대 주문 기록만 선택

In [151]:
식당2 = 식당.reset_index().set_index(['corner', 'dine_type'])
식당2 = 식당2.sort_index()
식당2[:3]

date         menu is_sold_out  use_count  \
corner       dine_type                                                  
after school dinner    2013-08-14         라면세트           F        114   
             dinner    2013-08-19  라면+롤밥 &고급음료           F        150   
             dinner    2013-08-20  라면+롤밥 &고급음료           F        155   

                        pred_count  additional  good  ok  bad  diff_use_pred  \
corner       dine_type                                                         
after school dinner              0           0     0   0    0           -114   
             dinner              0           0     0   0    0           -150   
             dinner              0           0     0   0    0           -155   

                        year  month  day  wday  
corner       dine_type                          
after school dinner     2013      8   14     2  
             dinner     2013      8   19     0  
             dinner     2013      8   20     1

In [155]:
아침주문 = 식당2.swaplevel(0,1).ix['breakfast']
아침주문 = 아침주문.set_index('date', append=True)
아침주문[['menu', 'use_count']][:3]

menu  use_count
corner        date                         
rice & soup 1 2013-08-06   설렁탕정식          1
              2013-08-12  뚝배기순대국         26
              2013-08-13  사골우거지탕         26

In [158]:
아침주문2 = 식당[식당.dine_type == 'breakfast']
아침주문2.set_index('corner', append=True).swaplevel(0,1)[:3]

dine_type    menu is_sold_out  use_count  \
corner        date                                                   
rice & soup 1 2013-08-06  breakfast   설렁탕정식           F          1   
              2013-08-12  breakfast  뚝배기순대국           F         26   
              2013-08-13  breakfast  사골우거지탕           F         26   

                          pred_count  additional  good  ok  bad  \
corner        date                                                
rice & soup 1 2013-08-06           0           0     0   0    0   
              2013-08-12           0           0     0   0    0   
              2013-08-13           0           0     0   0    0   

                          diff_use_pred  year  month  day  wday  
corner        date                                               
rice & soup 1 2013-08-06             -1  2013      8    6     1  
              2013-08-12            -26  2013      8   12     0  
              2013-08-13            -26  2013      8   13     1

## 데이터 변형

In [161]:
NC시즌 = NC시즌.fillna(method='ffill')

In [168]:
테나 = NC시즌.set_index('선수명').ix[['나성범', '테임즈']]

In [171]:
테나테나 = pd.concat([테나]*2)

In [173]:
테나테나.duplicated()

선수명
나성범    False
나성범    False
나성범    False
테임즈    False
테임즈    False
나성범     True
나성범     True
나성범     True
테임즈     True
테임즈     True
dtype: bool

In [175]:
테나테나[테나테나.duplicated()]

,시즌,타석,안타,홈런
선수명,,,,
나성범,2013.0,458,98,14
나성범,2014.0,536,157,30
나성범,2015.0,622,184,28
테임즈,2014.0,514,152,37
테임즈,2015.0,595,180,47


In [176]:
테나테나.drop_duplicates()

,시즌,타석,안타,홈런
선수명,,,,
나성범,2013.0,458,98,14
나성범,2014.0,536,157,30
나성범,2015.0,622,184,28
테임즈,2014.0,514,152,37
테임즈,2015.0,595,180,47


In [180]:
테나테나.reset_index().drop_duplicates(['선수명'], keep='last')

,선수명,시즌,타석,안타,홈런
7,나성범,2015.0,622,184,28
9,테임즈,2015.0,595,180,47


### 함수나 매핑을 이용해서 데이터 변형

In [181]:
식당.corner.value_counts()

rice & soup 1    1422
take out         1381
after school      929
grill & fry       925
noodle bar        923
burger&pizza      913
rice & soup 2     332
Name: corner, dtype: int64

In [185]:
분류사전 = {
    'rice & soup 1': '한식', 'rice & soup 2': '한식'
}

식당.corner.map(분류사전).fillna('기타').value_counts()

기타    5071
한식    1754
Name: corner, dtype: int64

In [187]:
날짜 = 식당.reset_index()['date']
날짜[:3]

0   2013-08-06
1   2013-08-06
2   2013-08-06
Name: date, dtype: datetime64[ns]

요일을 알고 싶다면?

In [188]:
날짜[0].year

2013

In [190]:
def 연도추출(날짜):
    return 날짜.year

날짜.map(연도추출)[:3]

0    2013
1    2013
2    2013
Name: date, dtype: int64

In [189]:
날짜.map(lambda 날짜: 날짜.year)[:3]

0    2013
1    2013
2    2013
Name: date, dtype: int64

In [193]:
식당.corner.replace(
    'rice & soup 1', '한식').value_counts()

한식               1422
take out         1381
after school      929
grill & fry       925
noodle bar        923
burger&pizza      913
rice & soup 2     332
Name: corner, dtype: int64

In [192]:
식당.corner.replace(
    ['rice & soup 1', 'rice & soup 2'], '한식').value_counts()

한식              1754
take out        1381
after school     929
grill & fry      925
noodle bar       923
burger&pizza     913
Name: corner, dtype: int64

In [194]:
식당.corner.replace(
    ['rice & soup 1', 'rice & soup 2'], 
    ['밥1', '밥2']
).value_counts()

밥1              1422
take out        1381
after school     929
grill & fry      925
noodle bar       923
burger&pizza     913
밥2               332
Name: corner, dtype: int64

In [195]:
식당.corner.replace(
    {
        'rice & soup 1': '밥1', 
        'burger&pizza': '버거',
        'rice & soup 2': '밥2'
    }
).value_counts()

밥1              1422
take out        1381
after school     929
grill & fry      925
noodle bar       923
버거               913
밥2               332
Name: corner, dtype: int64

### 축 라벨 변경

색인이나 열 제목 변경할 때 사용

In [198]:
작은식당 = 식당[['dine_type', 'corner', 'menu', 'use_count']]
작은식당[:3]

,dine_type,corner,menu,use_count
date,,,,
2013-08-06,dinner,grill & fry,훈제연어벤또(200개),213
2013-08-06,dinner,rice & soup 1,뚝배기 우거지 갈비탕,261
2013-08-06,lunch,noodle bar,판모밀정식,267


In [201]:
작은식당.rename(columns={        
        'dine_type': '시간대',
        'use_count': '주문수량'
    })[:3]

,시간대,corner,menu,주문수량
date,,,,
2013-08-06,dinner,grill & fry,훈제연어벤또(200개),213
2013-08-06,dinner,rice & soup 1,뚝배기 우거지 갈비탕,261
2013-08-06,lunch,noodle bar,판모밀정식,267


In [203]:
작은식당.rename(columns=str.upper)[:3]

,DINE_TYPE,CORNER,MENU,USE_COUNT
date,,,,
2013-08-06,dinner,grill & fry,훈제연어벤또(200개),213
2013-08-06,dinner,rice & soup 1,뚝배기 우거지 갈비탕,261
2013-08-06,lunch,noodle bar,판모밀정식,267


In [204]:
작은식당.rename(columns=lambda 제목: '**{}**'.format(제목))[:3]

,**dine_type**,**corner**,**menu**,**use_count**
date,,,,
2013-08-06,dinner,grill & fry,훈제연어벤또(200개),213
2013-08-06,dinner,rice & soup 1,뚝배기 우거지 갈비탕,261
2013-08-06,lunch,noodle bar,판모밀정식,267


In [205]:
프로그래머 = pd.read_csv('data/geeks.csv', encoding='utf8')

In [ ]:
깔끔한제목 = [제목.strip() for 제목 in 프로그래머.columns]
프로그래머.columns = 깔끔한제목

In [212]:
프로그래머 = 프로그래머.rename(columns=str.strip)

In [213]:
프로그래머['직업']

0     컴퓨터 프로그래머/데이터 과학자
1             컴퓨터 프로그래머
2                컴퓨터과학자
Name: 직업, dtype: object

In [215]:
식당.rename(index=lambda 날짜: 날짜.year)[:3]

,dine_type,corner,menu,is_sold_out,use_count,pred_count,additional,good,ok,bad,diff_use_pred,year,month,day,wday
date,,,,,,,,,,,,,,,
2013,dinner,grill & fry,훈제연어벤또(200개),F,213,0,0,0,0,0,-213,2013,8,6,1
2013,dinner,rice & soup 1,뚝배기 우거지 갈비탕,F,261,0,0,0,0,0,-261,2013,8,6,1
2013,lunch,noodle bar,판모밀정식,F,267,0,0,0,0,0,-267,2013,8,6,1


### 구간 나누기

In [217]:
연령 = pd.read_excel('data/ages.xlsx', squeeze=True)
연령[:3]

0    30
1    15
2    15
Name: 연령, dtype: int64

In [218]:
len(연령)

5000

In [ ]:
구간 = [0, 10, 20, 30, 40, 50, 150]

In [219]:
연령대 = pd.cut(연령, 구간)

In [220]:
연령대[:3]

0    (20, 30]
1    (10, 20]
2    (10, 20]
Name: 연령, dtype: category
Categories (6, object): [(0, 10] < (10, 20] < (20, 30] < (30, 40] < (40, 50] < (50, 150]]

In [221]:
연령대.value_counts()

(10, 20]     3080
(20, 30]      810
(30, 40]      606
(40, 50]      310
(50, 150]     194
(0, 10]         0
Name: 연령, dtype: int64

In [222]:
pd.cut(연령, 구간, right=False).value_counts()

[10, 20)     2998
[20, 30)      822
[30, 40)      622
[40, 50)      347
[50, 150)     211
[0, 10)         0
Name: 연령, dtype: int64

In [229]:
pd.cut(연령, 구간, 
       labels=['십대', '이십대','삼십대', 
               '사십대', '오십대', '그이상'], right=False).value_counts()

이십대    2998
삼십대     822
사십대     622
오십대     347
그이상     211
십대        0
Name: 연령, dtype: int64

구간의 개수 설정

In [235]:
pd.cut(연령, 4, precision=1).value_counts()

(14.9, 28.8]    3749
(28.8, 42.5]     833
(42.5, 56.2]     326
(56.2, 70]        92
Name: 연령, dtype: int64

In [240]:
pd.qcut(연령, 4)

ValueError: Bin edges must be unique: array([ 15.,  15.,  15.,  29.,  70.])

## 문자열

In [241]:
text = 'a,b,    guido'

In [242]:
text.split(',')

['a', 'b', '    guido']

In [245]:
[문자.strip() for 문자 in text.split(',')]

['a', 'b', 'guido']

In [246]:
러시아인형 = '(((<<<{{{**인형****>>>>}}})))'

In [247]:
러시아인형.strip('()<>{}*')

'인형'

In [248]:
사람들 = ['이', '김', '박']

In [249]:
','.join(사람들)

'이,김,박'

In [250]:
':'.join(사람들)

'이:김:박'

In [251]:
로또 = [1,2,3,4,5,6]

1-2-3-4-5-6

In [254]:
로또문자 = '-'.join(map(str,로또))

In [265]:
로또문자[-1::-2]

'654321'

#### 문자열 양식

In [266]:
프로필 = ['이성주', 'seongjoo@codebasic.io']

In [267]:
print('이름: ' + 프로필[0] + ', ' + '이메일: ' + 프로필[1])

이름: 이성주, 이메일: seongjoo@codebasic.io


In [268]:
출력양식 = '이름: {0}, 이메일: {1}'
출력양식.format(프로필[0], 프로필[1])

'이름: 이성주, 이메일: seongjoo@codebasic.io'

In [269]:
출력양식.format(*프로필)

'이름: 이성주, 이메일: seongjoo@codebasic.io'

In [270]:
신체지수 = {'키': 175, '몸무게': 88}

In [278]:
print('몸무게: {몸무게} kg, 키: {키} cm'.format(**신체지수))

몸무게: 88 kg, 키: 175 cm


In [280]:
'{0} {0:f} {0:.1f}'.format(3)

'3 3.000000 3.0'

In [283]:
잔고 = 1234567890
'{0:,}'.format(잔고)

'1,234,567,890'

In [284]:
'{0:02d}월 {1:02d}월'.format(1, 12)

'01월 12월'

In [288]:
식당.rename(
    index=lambda 날짜: '{}-{:02d}'.format(
        날짜.year, 날짜.month)).sort_index()[:10]

,dine_type,corner,menu,is_sold_out,use_count,pred_count,additional,good,ok,bad,diff_use_pred,year,month,day,wday
date,,,,,,,,,,,,,,,
2013-08,dinner,grill & fry,훈제연어벤또(200개),F,213,0,0,0,0,0,-213,2013,8,6,1
2013-08,dinner,rice & soup 1,뚝배기 우거지 갈비탕,F,261,0,0,0,0,0,-261,2013,8,6,1
2013-08,lunch,noodle bar,판모밀정식,F,267,0,0,0,0,0,-267,2013,8,6,1
2013-08,breakfast,rice & soup 1,설렁탕정식,F,1,0,0,0,0,0,-1,2013,8,6,1
2013-08,lunch,rice & soup 1,A. 부대찌개정식,F,376,0,0,0,0,0,-376,2013,8,6,1
2013-08,dinner,noodle bar,차슈라멘,F,179,0,0,0,0,0,-179,2013,8,12,0
2013-08,breakfast,rice & soup 1,뚝배기순대국,F,26,0,0,0,0,0,-26,2013,8,12,0
2013-08,lunch,rice & soup 1,A:누룽지장각백숙,F,504,0,0,0,0,0,-504,2013,8,12,0
2013-08,lunch,noodle bar,유니자장면,F,167,0,0,0,0,0,-167,2013,8,12,0


### 정규표현식

In [293]:
고객의견 = """여차저차해서여, 연락주세요 
010-1234-5678. 감사합니다.
혹시, 통화가 안되면 010-2323-8282로 주세요.
"""

전화번호만 추출하고 싶다면

In [290]:
import re

In [ ]:
전화번호패턴 = re.compile('\d{3}-\d{4}-\d{4}')

In [294]:
전화번호패턴.findall(고객의견)

['010-1234-5678', '010-2323-8282']

In [295]:
앨런튜링="""앨런 매티슨 튜링(영어: Alan Mathison Turing, OBE, FRS, 1912년 6월 23일 ~ 1954년 6월 7일)은 영국의 수학자, 암호학자, 논리학자이자 컴퓨터 과학의 선구적 인물이다. 알고리즘과 계산 개념을 튜링 기계라는 추상 모델을 통해 형식화함으로써 컴퓨터 과학의 발전에 지대한 공헌을 했다.[2][3][4] 튜링 테스트의 고안으로도 유명하다. ACM에서 컴퓨터 과학에 중요한 업적을 남긴 사람들에게 매년 시상하는 튜링상은 그의 이름을 따 제정한 것이다. 이론 컴퓨터 과학과 인공지능 분야에 지대한 공헌을 했기 때문에 "컴퓨터 과학의 아버지"라고 불린다."""

#### 연습

yyyy년 mm월 dd일 형식의 날짜 추출

In [296]:
날짜패턴 = re.compile('\d{4}년 \d{1,2}월 \d{1,2}일')

In [297]:
날짜패턴.findall(앨런튜링)

['1912년 6월 23일', '1954년 6월 7일']

### pandas 벡터 단위 문자열 처리

"벡터 단위"라는 말은 곧 시리즈 단위로 처리한다는 얘기입니다.

In [300]:
날씨.weather.str.contains('눈|비')[:3]

0    False
1     True
2    False
Name: weather, dtype: bool

In [303]:
식당.dine_type.str.upper().str[:3]

date
2013-08-06    DIN
2013-08-06    DIN
2013-08-06    LUN
Name: dine_type, dtype: object

In [311]:
날씨.weather.str.split('/').str[:2][:3]

0       [구름많음]
1     [흐림, 안개]
2    [흐림, 소나기]
Name: weather, dtype: object

In [ ]:
인물.서술.str.findall(날짜패턴)

## 농무부 자료

In [ ]:
import json

In [315]:
db = json.load(open('data/pydata-book/foods-2011-10-03.json'))

In [316]:
info = DataFrame(
    db, columns=['description', 'group', 'id', 'manufacturer'])

In [317]:
nutrients = []
for rec in db:
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)
    
nutrients = pd.concat(nutrients, ignore_index=True)

In [318]:
nutrients.duplicated().sum()

14179

In [319]:
nutrients = nutrients.drop_duplicates()

In [320]:
nutrients.duplicated().sum()

0

In [321]:
info[:3]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,


In [322]:
nutrients[:3]

,description,group,units,value,id
0,Protein,Composition,g,25.18,1008
1,Total lipid (fat),Composition,g,29.20,1008
2,"Carbohydrate, by difference",Composition,g,3.06,1008


In [324]:
info = info.rename(
    columns={'description': '식품명', 'group': '식품군'})
info[:3]

,식품명,식품군,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,


In [326]:
nutrients = nutrients.rename(
    columns={'description': '영양소', 'group': '영양소군'})
nutrients[:3]

,영양소,영양소군,units,value,id
0,Protein,Composition,g,25.18,1008
1,Total lipid (fat),Composition,g,29.20,1008
2,"Carbohydrate, by difference",Composition,g,3.06,1008


In [327]:
ndata = pd.merge(nutrients, info, on='id', how='outer')

In [333]:
ndata[['식품명', '영양소', 'value', 'units']][:3]

,식품명,영양소,value,units
0,"Cheese, caraway",Protein,25.18,g
1,"Cheese, caraway",Total lipid (fat),29.20,g
2,"Cheese, caraway","Carbohydrate, by difference",3.06,g


영양소별 식품별 함유량

In [343]:
영양소군별식품별 = ndata.groupby(['영양소', '식품군'])

"식품군 중 단백질 함량이 높은 것 상위 3개"

In [344]:
영양소군별식품별['value'].mean()['Protein'].sort_values(
    ascending=False)[:3]

식품군
Beef Products                    24.763560
Poultry Products                 23.469569
Lamb, Veal, and Game Products    23.424667
Name: value, dtype: float64